In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

path="/Users/xing/Desktop/test/"
filename=path+"store_id_relation.csv"
air_relation=pd.read_csv(filename)
filename=path+"air_reserve.csv"
air_reserve=pd.read_csv(filename)
filename=path+"hpg_reserve.csv"
hpg_reserve=pd.read_csv(filename)
filename=path+"air_store_info.csv"
air_store=pd.read_csv(filename)
filename=path+"hpg_store_info.csv"
hpg_store=pd.read_csv(filename)
filename=path+"air_visit_data.csv"
air_visits=pd.read_csv(filename)
# address air and hpg store data, merge hpg store data into air_store
stores = pd.merge(air_store,air_relation, how='left', on='air_store_id')
stores = pd.merge(stores, hpg_store, how='left', on='hpg_store_id', suffixes=['_air', '_hpg'])
stores.to_csv(path+'stores.csv', index=False)
#generate test data,sample_submission.csv is our test data set
filename=path+"sample_submission.csv"
stores_to_pred = pd.read_csv(filename)
#make id_data lables
id_split = stores_to_pred['id'].str.split('_', expand=True)
id_split.columns = ['source', 'store_id', 'visit_date']
stores_to_pred = pd.concat([stores_to_pred, id_split], axis=1)
#get new file with store_id_date column
stores_to_pred['air_store_id']=stores_to_pred['source'].str.cat(stores_to_pred['store_id'], sep='_')
stores_to_pred = stores_to_pred[['id', 'air_store_id', 'visit_date', 'visitors']]
stores_to_pred['visit_date'] = pd.to_datetime(stores_to_pred['visit_date'])
#merge air_visits data (past)to the whole data set,stores_to_pred is future data, which need to be predicted
FullData = pd.concat([air_visits, stores_to_pred], axis=0, keys=['past','predict'], names=['dataset']).reset_index(level='dataset')\
        .reset_index(drop=True)
#FullData=air_visits
FullData.to_csv(path+'full.csv', index=False)
stores_to_pred.to_csv(path+'resanrant_predict.csv', index=False)
#merge air_reserve and hgp_reserve data
hpg_reserve = pd.merge(hpg_reserve, air_relation, on='hpg_store_id')[air_reserve.columns]
reserves = pd.concat([air_reserve, hpg_reserve], axis=0)
reserves.to_csv(path+'reserves.csv', index=False)
FullData.head()


,dataset,air_store_id,id,visit_date,visitors
0,past,air_ba937bf13d40fb24,NaN,2016-01-13,25
1,past,air_ba937bf13d40fb24,NaN,2016-01-14,32
2,past,air_ba937bf13d40fb24,NaN,2016-01-15,29
3,past,air_ba937bf13d40fb24,NaN,2016-01-16,22
4,past,air_ba937bf13d40fb24,NaN,2016-01-18,6


In [2]:
path="/Users/xing/Desktop/test/"
filename=path+"full.csv"
FullData = pd.read_csv(filename, parse_dates=['visit_date'])
FullData['weekday'] = FullData.visit_date.dt.dayofweek
FullData['year'] = FullData.visit_date.dt.year
FullData['month'] = FullData.visit_date.dt.month
FullData['day_of_year'] = FullData.visit_date.dt.dayofyear
FullData['days_in_month'] = FullData.visit_date.dt.days_in_month
FullData['week_of_year'] = FullData.visit_date.dt.weekofyear

startday=FullData.visit_date.min()
first_day = pd.to_datetime(startday)
FullData['days_since_startday'] = (FullData.visit_date - first_day).apply(lambda dt: dt.days)
#merege holiday data
filename=path+"date_info.csv"
holiday_info = pd.read_csv(filename,  parse_dates=['calendar_date'])
holiday_info['tomorrow_is_holiday'] = holiday_info.holiday_flg.shift(-1).fillna(0).astype(int)
holiday_info['yesterday_is_holiday'] = holiday_info.holiday_flg.shift(1).fillna(0).astype(int)
FullData = pd.merge(left=FullData, right=holiday_info.drop('day_of_week', axis=1), how='left',left_on='visit_date', right_on='calendar_date')\
        .drop('calendar_date', axis=1)
#merge store_infomation , like city, stree, location, area
filename = path+"stores.csv"
stores = pd.read_csv(filename)
area_split = stores.air_area_name.str.split(' ', expand=True)
stores['area'] = area_split[0]
stores['city'] = area_split[1]
stores['street'] = area_split.iloc[:, 2:].apply(lambda row: ' '.join(row.dropna()), axis=1)
stores_by_street = stores.groupby(['air_area_name']).size().to_frame(name='stores_same_street').reset_index()
stores_by_city = stores.groupby(['area', 'city']).size().to_frame(name='stores_same_city').reset_index()
stores_by_area = stores.groupby('area').size().to_frame(name='stores_same_area').reset_index()
stores = pd.merge(left=stores, right=stores_by_street, how='left', on='air_area_name')
stores = pd.merge(left=stores, right=stores_by_city, how='left', on=['area', 'city'])
stores = pd.merge(left=stores, right=stores_by_area, how='left', on='area')
FullData = pd.merge(left=FullData, right=stores, how='left', on='air_store_id')
#merge reserves data 
filename = path+"reserves.csv"
reserves = pd.read_csv(filename, parse_dates=['visit_datetime', 'reserve_datetime'])
reserves['planned_visit_date'] = reserves.visit_datetime.dt.date.astype('datetime64[ns]')
reserves['reserve_date'] = reserves.reserve_datetime.dt.date
reserves['reserve_ahead'] = reserves.visit_datetime - reserves.reserve_datetime.astype('datetime64[ns]')
reserves['days_ahead'] = reserves.reserve_ahead.apply(lambda delta_t: delta_t.days)
reserve_summary = reserves.groupby(['air_store_id', 'planned_visit_date'])['reserve_visitors', 'days_ahead']\
                        .agg({'reserve_visitors': ['count','sum'], 'days_ahead': 'mean'})\
                         .reset_index()
reserve_summary.columns=['air_store_id', 'planned_visit_date', 'reserves', 'reserve_visitors', 'reserve_avg_days_ahead']
reserve_summary.head()
FullData = pd.merge(left=FullData, right=reserve_summary, how='left', 
                left_on=['air_store_id', 'visit_date'], 
                right_on=['air_store_id', 'planned_visit_date'])

FullData.head()


/Users/xing/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,dataset,air_store_id,id,visit_date,visitors,weekday,year,month,day_of_year,days_in_month,...,area,city,street,stores_same_street,stores_same_city,stores_same_area,planned_visit_date,reserves,reserve_visitors,reserve_avg_days_ahead
0,past,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,...,Tōkyō-to,Minato-ku,Shibakōen,51,61,444,NaT,NaN,NaN,NaN
1,past,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,...,Tōkyō-to,Minato-ku,Shibakōen,51,61,444,NaT,NaN,NaN,NaN
2,past,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,...,Tōkyō-to,Minato-ku,Shibakōen,51,61,444,NaT,NaN,NaN,NaN
3,past,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,...,Tōkyō-to,Minato-ku,Shibakōen,51,61,444,NaT,NaN,NaN,NaN
4,past,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,...,Tōkyō-to,Minato-ku,Shibakōen,51,61,444,NaT,NaN,NaN,NaN


In [3]:
#merge important feature column
def MergeFeatures(df, colname1, colname2, sep='_'):
    featureMerge = df[colname1].astype(str).str.cat(df[colname2].astype(str), sep=sep)
    return featureMerge
#merge important feature into one column---area,weekday,holiday_flag, location important
FullData['area_genre'] = MergeFeatures(FullData, 'air_area_name', 'air_genre_name')
FullData['store_weekday'] = MergeFeatures(FullData, 'air_store_id', 'weekday')
FullData['store_weekday_holiday'] = MergeFeatures(FullData, 'store_weekday', 'holiday_flg')

#group by air_store_id, weekday and holiday_flg,so we can averge number of vistor every week of the same resaurants
group_by_cols = ['air_store_id', 'weekday', 'holiday_flg']
visitor_stats = FullData.query('dataset == "past"').groupby(group_by_cols)['visitors'].agg(['mean', 'median', 'min', 'max'])\
                .rename(columns=lambda colname: str(colname)+'_visitors')\
                .reset_index()

FullData = FullData.merge(visitor_stats, how='left', on=group_by_cols)
#caculate number of vistors per week per air_store 
FullData['store_week_of_year'] = MergeFeatures(FullData, 'air_store_id', 'week_of_year')
group_bycol_week=['air_store_id','year','week_of_year','holiday_flg']
vistor_week=FullData.query('dataset == "past"').groupby(group_bycol_week)['visitors'].agg(['mean', 'median','min','max'])\
                .rename(columns=lambda colname: str(colname)+'_week_visitors')\
                .reset_index()

print vistor_week.head()
FullData = FullData.merge(vistor_week, how='left', on=group_bycol_week)

#lable friday and saturay as holiday
FullData.loc[FullData['weekday']==4,'holiday_flg']=1
FullData.loc[FullData['weekday']==5,'holiday_flg']=1

#caculate number of visitor per month per year per store
FullData['store_month_of_year'] = MergeFeatures(FullData, 'air_store_id', 'week_of_year')
group_bycol_month=['air_store_id','year','month']
vistor_month=FullData.query('dataset == "past"').groupby(group_bycol_month)['visitors'].agg(['mean', 'median','min','max'])\
                .rename(columns=lambda colname: str(colname)+'_month_visitors')\
                .reset_index()
#print vistor_week.head()
FullData = FullData.merge(vistor_month, how='left', on=group_bycol_month)

FullData.to_csv(path+'full_train.csv', index=False)
FullData.head()
#print FullData.groupby(group_by_cols)['visitors'].agg(['min','mean','max'])

           air_store_id  year  week_of_year  holiday_flg  mean_week_visitors  \
0  air_00a91d42b08b08d9  2016            26            0           22.000000   
1  air_00a91d42b08b08d9  2016            27            0           26.833333   
2  air_00a91d42b08b08d9  2016            28            0           24.333333   
3  air_00a91d42b08b08d9  2016            29            0           32.000000   
4  air_00a91d42b08b08d9  2016            30            0           23.500000   

   median_week_visitors  min_week_visitors  max_week_visitors  
0                  22.0                  9                 35  
1                  27.0                 11                 42  
2                  26.5                  5                 36  
3                  25.0                 16                 53  
4                  22.0                 16                 34  


,dataset,air_store_id,id,visit_date,visitors,weekday,year,month,day_of_year,days_in_month,...,store_week_of_year,mean_week_visitors,median_week_visitors,min_week_visitors,max_week_visitors,store_month_of_year,mean_month_visitors,median_month_visitors,min_month_visitors,max_month_visitors
0,past,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,...,air_ba937bf13d40fb24_2,27.0,27.0,22.0,32.0,air_ba937bf13d40fb24_2,20.5,21.5,6.0,32.0
1,past,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,...,air_ba937bf13d40fb24_2,27.0,27.0,22.0,32.0,air_ba937bf13d40fb24_2,20.5,21.5,6.0,32.0
2,past,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,...,air_ba937bf13d40fb24_2,27.0,27.0,22.0,32.0,air_ba937bf13d40fb24_2,20.5,21.5,6.0,32.0
3,past,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,...,air_ba937bf13d40fb24_2,27.0,27.0,22.0,32.0,air_ba937bf13d40fb24_2,20.5,21.5,6.0,32.0
4,past,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,...,air_ba937bf13d40fb24_3,18.5,19.5,6.0,31.0,air_ba937bf13d40fb24_3,20.5,21.5,6.0,32.0
